# Shakespeare in a graph

get the data from the Guttenbenrg Corpus

http://www.gutenberg.org/cache/epub/100/100.txt

rename the data and run Word2vec CBOW once the Sahakespeare text has been binarized

 ### Python Code

In [2]:
from neo4jrestclient import client
from neo4jrestclient.client import GraphDatabase
db = GraphDatabase("http://localhost:7474","neo4j","valy1990")
#db = GraphDatabase("http://localhost:7474","neo4j","xavier")
from gensim.models import word2vec
import re
from neo4jrestclient.client import GraphDatabase
import os 
#cwd =os.getcwd()


C:\Anaconda2\envs\python3Env\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:
mypath = os.getcwd()
pathData = mypath

In [4]:
class MySentences(object):
    def __init__(self, dirname, filename):
        self.dirname = dirname
        self.filename = filename
 
    def __iter__(self):
        for line in open(os.path.join(self.dirname, self.filename)):
            yield line.split()

In [5]:
sentencesShakespeare = MySentences(pathData,'shakespeare.txt')

In [27]:
model = word2vec.Word2Vec(sentencesShakespeare, min_count=50, size=200, sg=0,cbow_mean=1)

In [28]:
model.wv['the']
# or model['the']

array([ 0.10318378, -0.01143399, -0.40354472,  0.14453769,  0.18503197,
       -0.26849663, -0.41398111,  0.20695838,  0.20380649, -0.33432022,
        0.07091946, -0.21679181,  0.22540955,  0.00686741, -0.36712229,
       -0.52191943,  0.46489534,  0.33875161,  0.51797944,  0.7633487 ,
       -0.68542451, -0.05233897, -0.02719302,  0.07380115,  0.25989929,
       -0.42372516, -0.21933682,  0.3463186 ,  0.08699068, -0.13983421,
        0.38972062, -0.34776199, -0.28163522,  0.72355032,  0.14265998,
        0.18329003,  0.28499898, -0.24053566,  0.09843031, -0.05217484,
        0.05126661, -0.14381196, -0.07725375, -0.33922732,  0.22646734,
        0.3341735 ,  0.54713142,  0.14259708,  0.42431283,  0.44369584,
        0.02754814,  0.09515078, -0.02341047,  0.20675211,  0.43211755,
       -0.01391798, -0.08355758,  0.09801952,  0.04709624,  0.22624503,
       -0.23171608, -0.09650106, -0.23638722,  0.68651175, -0.47933167,
        0.31967154,  0.37850273,  0.09487572,  0.0259275 , -0.33

### Recursive Function

In [26]:
cachedWords = {}
 
## settings
rootWord = 'love'   # Initial Node
width = 5         # Width is the number of similar words to our intial node word
depth = 4           # How deep do we go in the graph
 
## Create all the labels

wordLabels = []
for i in range(1, depth+2):
    wordLabels.append(db.labels.create("WordLevel"+str(i)) )
 
## recursive function to find words ranked by cosine distance adding them to the Database
def process_word(thisWord, thisDepth):
    # Add the word to the DB
    wordLabels[thisDepth].add(thisWord)
    # Find similar words
    words = model.similar_by_word(thisWord.get(key='title'), topn=width)
 
    # Parse the similar words
    nextDepth = thisDepth + 1
    for word in words:
        # Creates key to track if word exists in DB to keep the graph clean
        key = re.sub('\W+', '', word[0]).lower()
 
        # Add word to DB
        if key not in cachedWords:
            dbWordNode = db.nodes.create(title=word[0])
            cachedWords[key] = dbWordNode
            dbWordNode.IS_SIMILAR(thisWord)
 
            # Check for depth
            if(nextDepth < depth):
                process_word(dbWordNode, nextDepth)
            else:
                wordLabels[nextDepth].add(dbWordNode)
 
      
        else:
            dbWordNode = cachedWords[key]
            if(dbWordNode is not thisWord):
                dbWordNode.IS_SIMILAR(thisWord)
 
 
 

cachedWords[rootWord] = db.nodes.create(title=rootWord)
process_word(cachedWords[rootWord], 0)